In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import json
import requests

In [2]:
def input_data(driver,class_O): #driver과 필터값을 주면 해당필터의 목록을 보여주고
                                #값을 받아 해당 카테고리로 이동하는 함수
    main_url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&includeMen=false&priceMin=&priceMax="
    
    
    if class_O == "기간":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[1]/dl/dd/ul"""
    elif class_O == "대분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
    elif class_O == "중분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""
    elif class_O == "가격":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[4]/dl/dd/ul"""
    else:
        print("잘못된 input_data함수 사용")
        return 0
    
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.TAG_NAME,"a")
    print("입력가능 항목")
    word = []

    for date in data_list:
        word.append(date.get_attribute("innerText"))
    print(word)
    input_date = input("원하시는 항목을 입력해 주십시오\n")

    try:
        if input_date in word:

            data_list[word.index(input_date)].click()
            
        else:
            raise TypeError 
    except TypeError:
        print("입력가능한 항목이 아닙니다.")
    
def crawling_data(driver):
    xpath = """//*[@id="goodsRankList"]"""
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.CLASS_NAME,"article_info")







In [3]:
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/home/rds/amr_ws/edm/data",
        "download.prompt_for_download":False}
options.add_experimental_option("prefs",prefs)
url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
driver = webdriver.Chrome(service=Service("../../driver/chromedriver-linux64/chromedriver"),
                        options=options)
driver.get(url)
driver.maximize_window()

#직접 유저가 넣는것에서 바뀌면 사이에 대기시간이 요구될 수 있음
input_data(driver, "기간")
input_data(driver, "대분류")
input_data(driver,"중분류")
input_data(driver, "가격")








입력가능 항목
['실시간', '일간', '주간', '월간', '3개월']
입력가능 항목
['상의', '아우터', '바지', '원피스', '스커트', '스니커즈', '신발', '가방', '여성 가방', '스포츠/용품', '모자', '양말/레그웨어', '속옷', '선글라스/안경테', '액세서리', '시계', '주얼리', '뷰티', '디지털/테크', '리빙', '컬처', '반려동물']
입력가능 항목
['후드 집업', '블루종/MA-1', '레더/라이더스 재킷', '무스탕/퍼', '트러커 재킷', '슈트/블레이저 재킷', '카디건', '아노락 재킷', '플리스/뽀글이', '트레이닝 재킷', '스타디움 재킷', '환절기 코트', '겨울 싱글 코트', '겨울 더블 코트', '겨울 기타 코트', '롱패딩/롱헤비 아우터', '숏패딩/숏헤비 아우터', '패딩 베스트', '베스트', '사파리/헌팅 재킷', '나일론/코치 재킷', '기타 아우터']
입력가능 항목
['5만원 이하', '5~10만원', '10~20만원', '20~30만원', '30만원 이상']


In [67]:
####################

xpath = """//*[@id="goodsRankList"]"""
data_bar = driver.find_element(By.XPATH, xpath)
data_list = data_bar.find_elements(By.CLASS_NAME,"list_img")
product_link = []
product_names = []
product_brand = []
product_gender = []
product_sell = []
product_goods = []
product_review_count = []
product_member_price = []
product_nomal_price = []
product_tags = []
product_bigCategory = []
product_smallCategory = []

goods_tmp = driver.find_element(By.ID,"goodsRankList")
goods_like = goods_tmp.find_elements(By.NAME,"count")

for like in goods_like:
    product_goods.append(like.get_attribute("innerText"))

for data in data_list:
    link = (data.find_element(By.TAG_NAME,"a").get_attribute("href"))
    product_link.append(link)
    re = Request(url=link, headers={"User-Agent":generate_user_agent()})
    rel = requests.get(link)
    page = urlopen(re)
    soup = BeautifulSoup(page, "html.parser")
    product_names.append(soup.find(id="notexist_title").string)
    
    script_org = soup.find_all("script")
    script = script_org[-1].get_text()#
    script_low = (script[script.find("product.state = {")+len("product.state = {")-1:script.rfind(";")+1])
    val = eval(script_low[:script_low.rfind("}")+1].replace("false","False").replace("null","None").replace("true","True"))
    try:
        product_sell.append(val["differenceColorGoods"][0]["saleRate"])
    except IndexError:
        product_sell.append(None)
    
    product_brand.append(val["brand"])
    product_gender.append(val["sex"])
    product_bigCategory.append(val["category"]["categoryDepth1Title"])
    product_smallCategory.append(val["category"]["categoryDepth2Title"])
    product_review_count.append(val["goodsReview"]["totalCount"])
    product_member_price.append(val["goodsPriceMemberLevels"][0]["price"])
    product_nomal_price.append(val["goodsPrice"]["originPrice"])
    product_tags.append(str(val["goodsTags"]))

In [68]:
df = pd.DataFrame()
df["링크"] = product_link
df["이름"] = product_names
df["브랜드"] = product_brand
df["성별"] = product_gender
df["누적판매량"] = product_sell
df["대분류"] = product_bigCategory
df["중분류"] = product_smallCategory
df["좋아요수"] = product_goods
df["리뷰개수"] = product_review_count
df["회원가"] = product_member_price
df["일반가"] = product_nomal_price
df["태그"] = product_tags

In [69]:
df

,링크,이름,브랜드,성별,누적판매량,대분류,중분류,좋아요수,리뷰개수,회원가,일반가,태그
0,https://www.musinsa.com/app/goods/3135186?loc=...,Soki Zip Jacket Charcoal,yeseyesee,[남성],70.0,아우터,트러커 재킷,"1,032",27,47700,159000,"['자켓', '재킷', '코튼자켓', '집업', '23S\\/S', '셋업아이템']"
1,https://www.musinsa.com/app/goods/2713778?loc=...,ARCH LOGO REVERSIBLE CORDUROY TRUCKER JACKET NAVY,yale,"[남성, 여성]",75.0,아우터,트러커 재킷,"5,061",135,49900,199000,"['숏자켓', '재킷', '블루종', '트러커자켓', '코듀로이', '헤비아우터컬렉..."
2,https://www.musinsa.com/app/goods/3601701?loc=...,(우먼) 크롭 스웨이드 자켓_웜그레이,codegraphy,[여성],70.0,아우터,트러커 재킷,701,5,32700,109000,"['트러커', '트러커자켓', '간절기아우터', '가을아우터', '블루종', '스웨..."
3,https://www.musinsa.com/app/goods/3079270?loc=...,피니쉬 와셔 트러커 자켓_Black,lafudgestore,"[남성, 여성]",61.0,아우터,트러커 재킷,"1,159",35,29800,76000,['증정이벤트']
4,https://www.musinsa.com/app/goods/1278595?loc=...,에센셜 미니멀 트러커 자켓 (BLACK),trillion,[남성],78.0,아우터,트러커 재킷,"2,788",401,21195,109000,"['남자자켓', '오버핏', '바람막이', '베이직', '아우터', '집업', '바..."
...,...,...,...,...,...,...,...,...,...,...,...,...
85,https://www.musinsa.com/app/goods/3130469?loc=...,경량 데님 트러커 스카이 블루 (253339GP8Q),8seconds,[남성],NaN,아우터,트러커 재킷,76,14,41481,69900,['253339GP8Q']
86,https://www.musinsa.com/app/goods/3135183?loc=...,Soki Zip Jacket Beige,yeseyesee,[남성],70.0,아우터,트러커 재킷,290,4,47175,159000,"['자켓', '재킷', '코튼자켓', '집업', '23S\\/S', '셋업아이템']"
87,https://www.musinsa.com/app/goods/3135184?loc=...,Soki Zip Jacket Navy,yeseyesee,[남성],70.0,아우터,트러커 재킷,708,15,47700,159000,"['자켓', '재킷', '코튼자켓', '집업', '23S\\/S', '셋업아이템']"
88,https://www.musinsa.com/app/goods/3142235?loc=...,레이스업 콘트라스트 스티치 자켓 카키,partimento,[남성],50.0,아우터,트러커 재킷,639,21,39263,79400,"['아우터', '트러커자켓', '트러커', '빈티지', '자켓', '간절기아우터',..."


In [242]:
driver.close()